In [1]:
''' 
%%capture

#! pip install -U pip
! pip install TTS==0.8.0
! pip install bnnumerizer==0.0.2
! pip install bangla==0.0.2
! pip install gdown==4.5.1
! pip install bnunicodenormalizer==0.1.1
! pip install PyArabic==0.6.15
#! pip install pydub
! pip install Unidecode==1.3.6
#! pip install numpy
!pip install pydload==1.0.9

!pip install mishkal==0.4.1
!pip install codernitydb3==0.6.0
!pip install --upgrade --no-cache-dir gdown
!git clone https://github.com/ARBML/klaam
'''

!nvidia-smi

Fri Dec 16 01:45:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:65:00.0 Off |                  Off |
| 30%   36C    P8    28W / 300W |   7171MiB / 49140MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import bangla
import re
import soundfile as sf

from bnnumerizer import numerize

run_multilingual_tts = True


[Dataset link](https://www.kaggle.com/datasets/nazmuddhohaansary/i-hadith-all-books)

In [3]:
!ls ./ihadith/csvs/

100-mashhur-hadis.csv	daif-hadis-in-mishkat.csv  riadus-saalehin.csv
40-hadis.csv		hadis-somvar.csv	   romjaner-durbol-hadis.csv
abi-dawud.csv		ibn-majah.csv		   sahih-fajayele-amal.csv
adabul-mufrad.csv	jal-daif-hadis-serise.csv  sahih-hadise-kudsi.csv
al-lulu-wal-marjan.csv	juj-ul-rafaul-yadain.csv   shamayele-tirmidi.csv
bukhari.csv		mishkatul-masabih.csv	   silsila-sahiha.csv
bukhari_last_two.csv	muatta-malik.csv	   targib-wat-tahrib.csv
bulugul-maram.csv	muslim.csv		   tirmidi.csv
combined.csv		nasayi.csv		   upodesh.csv


In [4]:
hadis = pd.read_csv('./ihadith/csvs/romjaner-durbol-hadis.csv')
is_arabic = False
hadis = hadis.replace(np.nan, 'অনুপস্থিত।', regex=True)
dir_name = 'romjaner_durbol_hadis_wavs'
print(len(hadis))
hadis.head(3)

36


,book-name,hadith-no,chapter-name,header,preface,arabic,narrator,bangla,explanation,extra-note,validity
0,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,1,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,পরিচ্ছেদঃ,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,اللهم بارك لنا في رجب وشعبان وبلغنا رمضان .,বর্ণনাকারী থেকে বর্ণিতঃ,"রমযান কাছে এলে আমরা অনেকে নিম্নের দোয়াটি পড়ি, ...",অনুপস্থিত।,হাদিসের সনদ:\nএ হাদিসের সনদে দু’টি দোষ বা সমস্...,মুনকার
1,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,2,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,পরিচ্ছেদঃ,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,شهر رمضان أوله رحمه و أوسطه مغفرة و آخره عتق م...,বর্ণনাকারী থেকে বর্ণিতঃ,"“রমযান মাসের প্রথম অংশ রহমত, মধ্যম অংশ মাগফেরা...",অনুপস্থিত।,"দেখুন: “কিতাবুদ দুয়াফা” লিল উকাইলি: (২/১৬২), “...",মুনকার
2,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,3,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,পরিচ্ছেদঃ,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,مَنْ أَفْطَرَ يَوْمًا مِنْ رَمَضَانَ مِنْ غَيْ...,বর্ণনাকারী থেকে বর্ণিতঃ,“যে ব্যক্তি কোন কারণ ছাড়া রমযানের একদিন সওম ভঙ...,অনুপস্থিত।,ইমাম বুখারি তার সহিহ গ্রন্থে হাদিসটি টিকা হিসে...,দুর্বল হাদিস


In [5]:
print(len(hadis))
hadis['explanation'].isnull().mean() * 100

36


0.0

In [6]:
hadis['extra-note'].isnull().mean() * 100

0.0

In [7]:
hadis['extra-note'][3] != hadis['extra-note'][3]

False

In [8]:
hadis.head(2)

,book-name,hadith-no,chapter-name,header,preface,arabic,narrator,bangla,explanation,extra-note,validity
0,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,1,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,পরিচ্ছেদঃ,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,اللهم بارك لنا في رجب وشعبان وبلغنا رمضان .,বর্ণনাকারী থেকে বর্ণিতঃ,"রমযান কাছে এলে আমরা অনেকে নিম্নের দোয়াটি পড়ি, ...",অনুপস্থিত।,হাদিসের সনদ:\nএ হাদিসের সনদে দু’টি দোষ বা সমস্...,মুনকার
1,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,2,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,পরিচ্ছেদঃ,রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,شهر رمضان أوله رحمه و أوسطه مغفرة و آخره عتق م...,বর্ণনাকারী থেকে বর্ণিতঃ,"“রমযান মাসের প্রথম অংশ রহমত, মধ্যম অংশ মাগফেরা...",অনুপস্থিত।,"দেখুন: “কিতাবুদ দুয়াফা” লিল উকাইলি: (২/১৬২), “...",মুনকার


In [9]:
i=0
# bn_hadith = hadis.iloc[i]['book-name'] + ","+hadis.iloc[i]['header']+" ভূমিকা পড়ছি।"+hadis.iloc[i]['preface']+", অধ্যায়ের নাম "+hadis.iloc[i]['chapter-name'] + "।অধ্যায় থেকে " +  str(hadis.iloc[i]['hadith-no']) + " নম্বর হাদিস পাঠ করছি, হাদিসের মানঃ "+ hadis.iloc[i]['validity']+" যা বর্ণনা করেছিল "+hadis.iloc[i]['narrator']+'।'+hadis.iloc[i]['bangla']

bn_hadith = hadis.iloc[i]['book-name'] + ","+", অধ্যায়ের নাম "+hadis.iloc[i]['chapter-name'] + "।অধ্যায় থেকে " +  str(hadis.iloc[i]['hadith-no']) + " নম্বর হাদিস পাঠ করছি, হাদিসের মানঃ "+ hadis.iloc[i]['validity']+" যা বর্ণনা করেছিল "+hadis.iloc[i]['narrator']+'।'+hadis.iloc[i]['arabic']+hadis.iloc[i]['bangla']  
bn_hadith

'রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ,, অধ্যায়ের নাম রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ।অধ্যায় থেকে 1 নম্বর হাদিস পাঠ করছি, হাদিসের মানঃ মুনকার যা বর্ণনা করেছিল বর্ণনাকারী থেকে বর্ণিতঃ।اللهم بارك لنا في رجب وشعبان وبلغنا رمضان .রমযান কাছে এলে আমরা অনেকে নিম্নের দোয়াটি পড়ি, তবে এমন লোক খুব কম আছি যারা এর শুদ্ধতা ও অশুদ্ধতা সম্পর্কে অবগত-“হে আল্লাহ, আপনি রজব ও শাবান মাসে আমাদের জন্য বরকত রাখুন এবং আমাদেরকে রমযান পর্যন্ত পৌঁছার তাওফিক দিন”।'

In [10]:
unqiue_chapters = hadis['chapter-name'].unique()
print(len(unqiue_chapters))
unqiue_chapters

1


array(['রমযান বিষয়ে জাল ও দুর্বল হাদিসসমূহ'], dtype=object)

In [11]:
# stored = os.listdir(f'{dir_name}')
# stored

In [12]:
# broken_chapter = 'পোশাক-পরিচ্ছদ.wav'
def mlt_reader(hadis):
    bn_audio = []
#     curr_chapter = '১'
    for i in range(len(hadis)):
        ''' 
        chapter = hadis.iloc[i].header.split('/')
        chapter[0] = chapter[0].replace('.','')
        if(chapter[0] != curr_chapter):
            curr_chapter = chapter[0]
            bn_audio = []
        
        chapter_number = int(chapter[0])
        '''
        
        book = hadis.iloc[i]['chapter-name'].replace(' ','_').replace('/','_')
        book = re.sub('\u200c', '', book)
        
        is_this_file_exist = book+'.wav'
#         if(is_this_file_exist in stored):
#             continue
            
       
#         if(broken_chapter != is_this_file_exist):
#             continue
#         if(hadis.iloc[i]['hadith-no'] <= 5898):
#             continue
#         print("working on ",hadis.iloc[i]['hadith-no'], book)
        
#         book = re.sub(r'[\\*?,:()&"<>|\‘\’\”]',"_",book)
#         bn_hadith = hadis.iloc[i]['book-name'] + ","+hadis.iloc[i]['header']+" ভূমিকা পড়ছি।"+hadis.iloc[i]['preface']+", অধ্যায়ের নাম "+hadis.iloc[i]['chapter-name'] + "।অধ্যায় থেকে " +  str(hadis.iloc[i]['hadith-no']) + " নম্বর হাদিস পাঠ করছি, হাদিসের মানঃ "+ hadis.iloc[i]['validity']+" যা বর্ণনা করেছিল "+hadis.iloc[i]['narrator']+'।'+hadis.iloc[i]['bangla']
        if(is_arabic):
            if(('অনুপস্থিত' in str(hadis.iloc[i]['arabic'])) and ('অনুপস্থিত' in str(hadis.iloc[i]['preface']))):
                bn_hadith = str(hadis.iloc[i]['book-name']) + ","+", অধ্যায়ের নাম "+ str(hadis.iloc[i]['chapter-name']) + " অধ্যায় থেকে " +  str(hadis.iloc[i]['hadith-no']) + " নম্বর হাদিস পাঠ করছি, হাদিসের মানঃ "+ str(hadis.iloc[i]['validity'])+" যা বর্ণনা করেছিল "+ str(hadis.iloc[i]['narrator'])+ str(hadis.iloc[i]['bangla'])        
            elif('অনুপস্থিত' in str(hadis.iloc[i]['arabic'])):
                bn_hadith = str(hadis.iloc[i]['book-name']) + ","+", অধ্যায়ের নাম "+ str(hadis.iloc[i]['chapter-name']) + " অধ্যায় থেকে " +  str(hadis.iloc[i]['hadith-no']) + " নম্বর হাদিস পাঠ করছি, হাদিসের মানঃ "+ str(hadis.iloc[i]['validity'])+" যা বর্ণনা করেছিল "+ str(hadis.iloc[i]['narrator'])+ str(hadis.iloc[i]['bangla'])        

            elif('অনুপস্থিত' in str(hadis.iloc[i]['preface'])):
                bn_hadith = str(hadis.iloc[i]['book-name']) + ","+", অধ্যায়ের নাম "+ str(hadis.iloc[i]['chapter-name']) + " অধ্যায় থেকে " +  str(hadis.iloc[i]['hadith-no']) + " নম্বর হাদিস পাঠ করছি, হাদিসের মানঃ "+ str(hadis.iloc[i]['validity'])+" যা বর্ণনা করেছিল "+ str(hadis.iloc[i]['narrator'])+ str(hadis.iloc[i]['arabic'])+" "+ str(hadis.iloc[i]['bangla'])        
            else:
                bn_hadith = str(hadis.iloc[i]['book-name']) +" ভূমিকা পড়ছি।"+hadis.iloc[i]['preface']+ ","+", অধ্যায়ের নাম "+ str(hadis.iloc[i]['chapter-name']) + " অধ্যায় থেকে " +  str(hadis.iloc[i]['hadith-no']) + " নম্বর হাদিস পাঠ করছি, হাদিসের মানঃ "+ str(hadis.iloc[i]['validity'])+" যা বর্ণনা করেছিল "+ str(hadis.iloc[i]['narrator'])+ str(hadis.iloc[i]['arabic'])+" "+ str(hadis.iloc[i]['bangla'])
        
        else:
            if('অনুপস্থিত' in str(hadis.iloc[i]['preface'])):
                bn_hadith = str(hadis.iloc[i]['book-name']) + ","+", অধ্যায়ের নাম "+ str(hadis.iloc[i]['chapter-name']) + " অধ্যায় থেকে " +  str(hadis.iloc[i]['hadith-no']) + " নম্বর হাদিস পাঠ করছি, হাদিসের মানঃ "+ str(hadis.iloc[i]['validity'])+" যা বর্ণনা করেছিল "+ str(hadis.iloc[i]['narrator'])+ str(hadis.iloc[i]['bangla'])        
            else:
                bn_hadith = str(hadis.iloc[i]['book-name']) +" ভূমিকা পড়ছি।"+hadis.iloc[i]['preface']+ ","+", অধ্যায়ের নাম "+ str(hadis.iloc[i]['chapter-name']) + " অধ্যায় থেকে " +  str(hadis.iloc[i]['hadith-no']) + " নম্বর হাদিস পাঠ করছি, হাদিসের মানঃ "+ str(hadis.iloc[i]['validity'])+" যা বর্ণনা করেছিল "+ str(hadis.iloc[i]['narrator'])+ str(hadis.iloc[i]['bangla'])

        try:
          bn_hadith = bn_hadith.replace('/',' বাই ').replace('.',' ')
          bn_hadith = re.sub('হা/', ' হাদিস নম্বর ', bn_hadith)
          bn_hadith = re.sub('হাঃ', ' হাদিস নম্বর ', bn_hadith)
          bn_hadith = bn_hadith.strip()
        except:
          #debug
          print("error during processing ", i)
        
        audio=MLT_TTS(bn_hadith,resample_audio_to_out_sample_rate=True)
        filepath = f'./{dir_name}/{book}.wav'
        if not os.path.isfile(filepath):
            print("chapter-name = ",book)
            bn_audio = []
        bn_audio.append(audio)
        full_chapter_audio=np.concatenate(bn_audio,axis=-1)
        sf.write(filepath, full_chapter_audio, 22050)

In [13]:
! git clone https://github.com/nipponjo/tts-arabic-tacotron2

os.chdir('./tts-arabic-tacotron2')
!git checkout e384a3b0229f11e66311680a5b4402218bd433fb # we need version 2
os.chdir('../')

fatal: destination path 'tts-arabic-tacotron2' already exists and is not an empty directory.
M	utils/__init__.py
HEAD is now at e384a3b Update preprocessing


In [14]:
!pwd

/home/ansary/Shabab/book_readers


In [15]:
%%writefile ./tts-arabic-tacotron2/utils/__init__.py
import sys
import yaml
try:
    from yaml import CLoader as Loader
except ImportError:
    from yaml import Loader


class DictConfig(object):
    """Creates a Config object from a dict 
       such that object attributes correspond to dict keys.    
    """

    def __init__(self, config_dict):
        self.__dict__.update(config_dict)

    def __str__(self):
        return '\n'.join(f"{key}: {val}" for key, val in self.__dict__.items())

    def __repr__(self):
        return self.__str__()


def get_custom_config(fname):
    with open(fname, 'r') as stream:
        config_dict = yaml.load(stream, Loader)
    config = DictConfig(config_dict)
    return config


def get_basic_config():
    return get_custom_config('./tts-arabic-tacotron2/configs/basic.yaml')


def get_config(fname):
    config = get_basic_config()
    custom_config = get_custom_config(fname)

    config.__dict__.update(custom_config.__dict__)
    return config


def read_lines_from_file(path, encoding='utf-8'):
    lines = []
    with open(path, 'r', encoding=encoding) as f:
        for line in f:
            lines.append(line.strip())
    return lines


def progbar(iterable, length=30, symbol='='):
    """Wrapper generator function for an iterable. 
       Prints a progressbar when yielding an item. \\
       Args:
          iterable: an object supporting iteration
          length: length of the progressbar
    """
    n = len(iterable)
    for i, item in enumerate(iterable):
        steps = length*(i+1) // n
        sys.stdout.write('\r')
        sys.stdout.write(f"[{symbol*steps:{length}}] {(100/n*(i+1)):.1f}%")
        if i == (n-1):
            sys.stdout.write('\n')
        sys.stdout.flush()
        yield item

Overwriting ./tts-arabic-tacotron2/utils/__init__.py


In [16]:

%%time

try:
    os.mkdir(f'{dir_name}')
except:
    print("exist!!")
    
if(run_multilingual_tts):
    from multilingual_tts_v2 import BigTextToAudio

    MLT_TTS=BigTextToAudio()

    mlt_reader(hadis)


/home/ansary/anaconda3/envs/mobassir/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


downloading bangla tts weights........
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
['<BLNK>', 'র', '<BLNK>', 'ম', '<BLNK>', 'য', '<BLNK>', 'া', '<BLNK>', 'ন', '<BLNK>', ' ', '<BLNK>', 'ব', '<BLNK>', 'ি', '<BLNK>', 'ষ', '<BLNK>', 'য়', '<BLNK>', 'ে', '<BLNK>', ' ', '<BLNK>', 'জ',

['<BLNK>', '(', '<BLNK>', 'দ', '<BLNK>', 'ে', '<BLNK>', 'খ', '<BLNK>', 'ু', '<BLNK>', 'ন', '<BLNK>', ':', '<BLNK>', ' ', '<BLNK>', 'স', '<BLNK>', 'ূ', '<BLNK>', 'র', '<BLNK>', 'া', '<BLNK>', ' ', '<BLNK>', 'ম', '<BLNK>', 'ু', '<BLNK>', 'জ', '<BLNK>', 'া', '<BLNK>', 'দ', '<BLNK>', 'া', '<BLNK>', 'ল', '<BLNK>', 'া', '<BLNK>', ':', '<BLNK>', 'ত', '<BLNK>', 'ি', '<BLNK>', 'ন', '<BLNK>', '-', '<BLNK>', 'চ', '<BLNK>', 'া', '<BLNK>', 'র', '<BLNK>', ')', '<BLNK>']
 [!] Character '(' not found in the vocabulary. Discarding it.
['<BLNK>', '(', '<BLNK>', 'দ', '<BLNK>', 'ে', '<BLNK>', 'খ', '<BLNK>', 'ু', '<BLNK>', 'ন', '<BLNK>', ':', '<BLNK>', ' ', '<BLNK>', 'স', '<BLNK>', 'ূ', '<BLNK>', 'র', '<BLNK>', 'া', '<BLNK>', ' ', '<BLNK>', 'ম', '<BLNK>', 'ু', '<BLNK>', 'জ', '<BLNK>', 'া', '<BLNK>', 'দ', '<BLNK>', 'া', '<BLNK>', 'ল', '<BLNK>', 'া', '<BLNK>', ':', '<BLNK>', 'ত', '<BLNK>', 'ি', '<BLNK>', 'ন', '<BLNK>', '-', '<BLNK>', 'চ', '<BLNK>', 'া', '<BLNK>', 'র', '<BLNK>', ')', '<BLNK>']
 [!] Character 

In [17]:
!nvidia-smi

Fri Dec 16 02:21:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:65:00.0 Off |                  Off |
| 30%   36C    P8    28W / 300W |   9194MiB / 49140MiB |     12%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
hadis[hadis['chapter-name'] == 'যাকাত']

,book-name,hadith-no,chapter-name,header,preface,arabic,narrator,bangla,explanation,extra-note,validity


In [19]:
len(hadis)

36

In [20]:
# hadis[hadis['hadith-no'] == 7276]